<a href="https://colab.research.google.com/github/Anson2Leung/ece405-assignment1-basics/blob/main/assignment_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!git clone https://github.com/Anson2Leung/ece405-assignment1-basics.git

fatal: destination path 'ece405-assignment1-basics' already exists and is not an empty directory.


In [2]:
# Install uv
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Add uv to the system path so you can use it directly
import os
os.environ['PATH'] = f"{os.path.expanduser('~/.cargo/bin')}:{os.environ['PATH']}"

downloading uv 0.9.28 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [3]:
%cd ece405-assignment1-basics

!uv pip install -r requirements.txt --system
!pip install "numpy<2.0" jaxtyping typeguard langsmith

/content/ece405-assignment1-basics
Using Python 3.12.12 environment at: /usr
Audited 3 packages in 108ms


# Test run

In [ ]:
!pytest tests/test_train_bpe.py

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0
rootdir: /content/ece405-assignment1-basics
configfile: pytest.ini
plugins: jaxtyping-0.3.7, anyio-4.12.1, typeguard-4.4.4, langsmith-0.6.6
collected 3 items                                                              

tests/test_train_bpe.py::test_train_bpe_speed PASSED                     [ 33%]
tests/test_train_bpe.py::test_train_bpe PASSED                           [ 66%]
tests/test_train_bpe.py::test_train_bpe_special_tokens FAILED            [100%]

=================================== FAILURES ===================================
________________________ test_train_bpe_special_tokens _________________________

snapshot = <tests.conftest.Snapshot object at 0x7d0bcca5ca70>

    def test_train_bpe_special_tokens(snapshot):
        """
        Ensure that the special tokens are added to the vocabulary and not
        merged with other tokens.


# Problem (train_bpe_tinystories): BPE Training on TinyStories (2 points)


In [ ]:
%cd ..
%mkdir -p data
%cd data

# !wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt
# !wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-valid.txt

# !wget https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_train.txt.gz
!gunzip owt_train.txt.gz
!wget https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_valid.txt.gz
!gunzip owt_valid.txt.gz

%cd ..
%cd ece405-assignment1-basics

/home/ansonl32/ECE405
/home/ansonl32/ECE405/data


In [6]:
import sys
import os
import json

%cd /content/ece405-assignment1-basics
sys.path.append('/content/ece405-assignment1-basics/assignment_files')
from bpe_tokenizer import train_bpe, save_vocab, save_merges

INPUT_PATH = "/content/data/TinyStoriesV2-GPT4-train.txt"
VOCAB_SIZE = 10000
SPECIAL_TOKENS = ["<|endoftext|>"]

vocab, merges = train_bpe(INPUT_PATH, VOCAB_SIZE, SPECIAL_TOKENS)

save_vocab(vocab, "TinyStories_vocab.json")
save_merges(merges, "TinyStories_merges.txt")

/content/ece405-assignment1-basics
Dataset:  /content/data/TinyStoriesV2-GPT4-train.txt
Number of processes requested:  2
Special tokens as bytes:  [b'<|endoftext|>']
Total number of tasks to complete:  1
Pre-tokenization complete. Unique pre-tokens: 47063
Number of merges to perform:  9743


Merging Vocab: 100%|██████████| 9743/9743 [00:36<00:00, 267.51it/s]


Pre-tokenize time complete in 472.79s
Initialization time complete in 0.95s
BPE Merges complete in 36.43s
Total training time: 510.17s
Total memory used: 81.02 MB
Vocab saved to TinyStories_vocab.json
Merges saved to TinyStories_merges.txt


In [7]:
def get_decoded_string(token_bytes):
    return token_bytes.decode('utf-8', errors='ignore')

# Find the longest token in the vocabulary
longest_token_id = max(vocab, key=lambda k: len(get_decoded_string(vocab[k])))
longest_token_bytes = vocab[longest_token_id]
longest_text = get_decoded_string(longest_token_bytes)

print(f"--- Analysis Results ---")
print(f"Longest Token ID: {longest_token_id}")
print(f"Longest Token Text: '{longest_text}'")
print(f"Character Length: {len(longest_text)}")

--- Analysis Results ---
Longest Token ID: 7107
Longest Token Text: ' accomplishment'
Character Length: 15


# Problem (train_bpe_expts_owt): BPE Training on OpenWebText (2 points



In [ ]:
import sys
import os
import json

%cd /content/ece405-assignment1-basics
sys.path.append('/content/ece405-assignment1-basics/assignment_files')
from bpe_tokenizer import train_bpe, save_vocab, save_merges

INPUT_PATH = "/content/data/owt_train.txt"
VOCAB_SIZE = 32000
SPECIAL_TOKENS = ["<|endoftext|>"]

vocab2, merges2 = train_bpe(INPUT_PATH, VOCAB_SIZE, SPECIAL_TOKENS)

save_vocab(vocab2, "OWT_vocab.json")
save_merges(merges2, "OWT_merges.txt")

/content/ece405-assignment1-basics
Dataset:  /content/data/owt_train.txt
Number of processes requested:  2
Special tokens as bytes:  [b'<|endoftext|>']
Total number of tasks to complete:  1


In [ ]:
# Analysis
def get_decoded_string(token_bytes):
    return token_bytes.decode('utf-8', errors='ignore')

# Find the longest token in the vocabulary
longest_token_id = max(vocab2, key=lambda k: len(get_decoded_string(vocab2[k])))
longest_token_bytes = vocab2[longest_token_id]
longest_text = get_decoded_string(longest_token_bytes)

print(f"--- Analysis Results ---")
print(f"Longest Token ID: {longest_token_id}")
print(f"Longest Token Text: '{longest_text}'")
print(f"Character Length: {len(longest_text)}")

# Problem (tokenizer_experiments): Experiments with tokenizers (4 points)

In [1]:
def get_sample_documents(filepath: str, n_samples: int = 10):
    samples = []
    separator = "<|endoftext|>"
    
    print(f"Sampling {n_samples} documents from {filepath}...")
    
    with open(filepath, 'r', encoding='utf-8') as f:
        # Read a large enough chunk to get 10 stories (TinyStories are short)
        # 100KB is plenty for 10 stories.
        content = f.read(100_000) 
        
        # Split by the special token
        raw_docs = content.split(separator)
        
        for doc in raw_docs:
            clean_doc = doc.strip() # Removes the '\n' at Line 1 and Line 6
            if clean_doc:
                samples.append(clean_doc)
            
            if len(samples) >= n_samples:
                break
                
    return samples[:n_samples]

In [2]:
import time
from assignment_files.tokenizer import Tokenizer

def analyze_compression(tokenizer: Tokenizer, documents: list[str], name: str):
    """
    Encodes documents and calculates compression ratio and throughput.
    """
    total_bytes = sum(len(d.encode('utf-8')) for d in documents)
    
    start_time = time.perf_counter()
    total_tokens = 0
    
    for token in tokenizer.encode_iterable(documents):
        total_tokens += 1
        
    duration = time.perf_counter() - start_time
    
    ratio = total_bytes / total_tokens if total_tokens > 0 else 0
    throughput = total_bytes / duration if duration > 0 else 0
    
    print(f"[{name}]")
    print(f"  Compression Ratio: {ratio:.2f} bytes/token")
    print(f"  Throughput:        {throughput / 1e6:.2f} MB/s")
    return ratio, throughput

In [3]:
import os
import numpy as np
import multiprocessing
from functools import partial
from tqdm import tqdm
from assignment_files.bpe_tokenizer import find_chunk_boundaries
from assignment_files.tokenizer import Tokenizer
from tqdm import tqdm

def _encode_worker(task_tuple, tokenizer):
    file_path, start, end = task_tuple
    
    with open(file_path, 'rb') as f:
        f.seek(start)
        chunk_bytes = f.read(end - start)
    
    # Decode bytes to string for the tokenizer
    text = chunk_bytes.decode('utf-8', errors='replace')
    
    # Encode using tokenizer
    tokens = tokenizer.encode(text)
    
    # Return as uint16
    return np.array(tokens, dtype=np.uint16)

def encode_dataset(tokenizer, input_filepath, output_filepath, procs=None):
    procs = procs or os.cpu_count()
    special_tokens_bytes = [t.encode('utf-8') for t in tokenizer.special_tokens]

    # find chunk boundries
    print(f"Calculating boundaries for {input_filepath}...")
    with open(input_filepath, 'rb') as f:
        boundaries = find_chunk_boundaries(f, procs * 4, special_tokens_bytes)

    # Task parameters (file_path, start_byte, end_byte)
    tasks = [(input_filepath, boundaries[i], boundaries[i+1]) 
             for i in range(len(boundaries) - 1)]

    # Process chunks in parallel and stream to a temporary binary file
    temp_bin = output_filepath + ".tmp"
    total_tokens = 0
    
    # Partial gives tokenizer instance to every worker
    worker_with_tokenizer = partial(_encode_worker, tokenizer=tokenizer)

    print(f"Encoding {len(tasks)} chunks using {procs} processes...")
    with open(temp_bin, 'wb') as f_out:
        with multiprocessing.Pool(processes=procs) as pool:
            # imap allows us to process results as they finish
            for token_arr in tqdm(pool.imap(worker_with_tokenizer, tasks), total=len(tasks)):
                f_out.write(token_arr.tobytes())
                total_tokens += len(token_arr)

    # Raw binary -> NumPy array -> .npy file
    print(f"Finalizing {total_tokens} tokens...")
    # We use mmap_mode if the file is truly massive, otherwise fromfile is fine
    final_data = np.fromfile(temp_bin, dtype=np.uint16)
    
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    np.save(output_filepath, final_data)
    
    # Cleanup
    if os.path.exists(temp_bin):
        os.remove(temp_bin)
        
    print(f"Completed! Saved to {output_filepath}")
    print(f"Final size: {final_data.nbytes / 1024**2:.2f} MB")

In [4]:
print("Initializing tokenizers...")

ts_tokenizer = Tokenizer.from_files(
    vocab_filepath="assignment_files/TinyStoriesVocab.json",
    merges_filepath="assignment_files/TinyStoriesMerges.json",
    special_tokens=["<|endoftext|>"]
)

owt_tokenizer = Tokenizer.from_files(
    vocab_filepath="assignment_files/OWTVocab.json",
    merges_filepath="assignment_files/OWTMerges.json",
    special_tokens=["<|endoftext|>"]
)

# Sample 10 documents
ts_samples = get_sample_documents("../data/TinyStoriesV2-GPT4-train.txt", n_samples=10)
owt_samples = get_sample_documents("../data/owt_train.txt", n_samples=10)



Initializing tokenizers...
Sampling 10 documents from ../data/TinyStoriesV2-GPT4-train.txt...
Sampling 10 documents from ../data/owt_train.txt...


In [17]:
# (a) Sample 10 documents from TinyStories and OpenWebText. Using your previously-trained TinyStories and OpenWebText tokenizers encode these 
# sampled documents into integer IDs. What is each tokenizer’s compression ratio (bytes/token)?
analyze_compression(ts_tokenizer, ts_samples, "TinyStories Tokenizer on TinyStories samples")
_, owt_throughput = analyze_compression(owt_tokenizer, owt_samples, "OWT Tokenizer on OWT Samples")

# (b)  What happens if you tokenize your OpenWebText sample with the TinyStories tokenizer? Compare the compression ratio and/or qualitatively describe what happens.
analyze_compression(ts_tokenizer, owt_samples, "TinyStories Tokenizer on OWT Samples")

# (c) Estimate the throughput of your tokenizer (e.g., in bytes/second). How long would it take to tokenize the Pile dataset (825GB of text)
pile_size_gb = 825
seconds_needed = (pile_size_gb * 1e9) / owt_throughput
print(f"\n[The Pile Estimation]")
print(f"  Time to tokenize 825GB: {seconds_needed / 3600:.2f} hours")

[TinyStories Tokenizer on TinyStories samples]
  Compression Ratio: 4.15 bytes/token
  Throughput:        0.96 MB/s
[OWT Tokenizer on OWT Samples]
  Compression Ratio: 4.69 bytes/token
  Throughput:        0.90 MB/s
[TinyStories Tokenizer on OWT Samples]
  Compression Ratio: 3.19 bytes/token
  Throughput:        0.31 MB/s

[The Pile Estimation]
  Time to tokenize 825GB: 253.95 hours


In [18]:

# (d) Using your TinyStories and OpenWebText tokenizers, encode the respective training and development datasets into a sequence of integer token ID
encode_dataset(ts_tokenizer, "../data/TinyStoriesV2-GPT4-train.txt", "../data/tinystories_train.npy")

Calculating boundaries for ../data/TinyStoriesV2-GPT4-train.txt...
Encoding 159 chunks using 40 processes...


100%|██████████| 159/159 [11:48<00:00,  4.45s/it] 


Finalizing 537848181 tokens...
Completed! Saved to assignment_files/tinystories_train.npy
Final size: 1025.86 MB


In [5]:
encode_dataset(owt_tokenizer, "../data/owt_train.txt", "../data/owt_train.npy")

Calculating boundaries for ../data/owt_train.txt...
Encoding 159 chunks using 40 processes...


100%|██████████| 159/159 [1:05:44<00:00, 24.81s/it] 


Finalizing 1810995702 tokens...
Completed! Saved to ../data/owt_train.npy
Final size: 3454.20 MB
